Tika cleanup plain text
===================

This notebook experiments with how to cleanup plain text and only extract the content that is relevant to the subject of a document. We aren't fully sure what we need to remove, but we want to look at relatively large PDF's and Word documents and see what we encounter.

I have looked at 20 learning materials. These were selected through the ``dataset_plaintext.py`` script.

Qualitative analysis
----------------------------

First we'll write the Tika output to file in order to easily read the output and compare the output side-by-side with the input files.

Then per file uuid we'll write down some conclusions on what we might do with the output.

In [8]:
import os
import json
from urllib.parse import urlparse

In [2]:
def load_resources(data):
    for resource in data:
        resource["content"] = json.loads(resource["fields"]["body"])
        resource["url"] = resource["fields"]["request"]["args"][0]

In [6]:
with open("http-tika-resources.plain.json", "r") as tika_resources:
    data = json.load(tika_resources)

load_resources(data)

In [9]:
for rsc in data:
    _, file_id = os.path.split(urlparse(rsc["url"]).path)
    with open(f"samples/document_plain/{file_id}.html", "w") as xml_file:
        xml_file.write(rsc["content"][0]["X-TIKA:content"])

#### 0b6a47dc-b50b-42a5-b7bb-2f6249cc4577
A presentation in PDF format where the slides are each in their own \<div class="page"\>. Texts are extracted rather literary with newlines included. So bulletpoint lists are not some sort of \<ul\>, but a dot character or a number followed by some text.

#### 1d863ed1-1609-453e-a6f3-5b5e107be543 
A research product in PDF format. Again the use of \<div class="page"\> elements. Links are annotated at the bottom of pages, but also remain inline. Headers are the first line of a \<p\>. Any text directly under the header follows in the same \<p\> as its header.

#### 8d5015a1-ace2-49ba-8c80-8e5c254b32cc
A PDF tutorial in three parts (this is part 2). Heading and footer text are added at the top of \<div class="page"\> elements. Link annotations are again placed at the bottom with the original link remaining unchanged inline.

#### 27f3d9a1-d752-4704-b6b8-0fe0439f0859
Again a PDF. It acts mostly like the PDF's before this. With footer text at the top of a page. The "links" in this PDF are actually only "blue underlined texts", but not clickable. The URL's in this plain text are not picked up in any way and there's a newline character half way one link. It will be tedious to extract links like this (because there is no delimiter).

#### 145cae9f-c834-4c38-8c34-4c76479da3ca
A word document created on an Apple. All text is wrapped in \<p\> tags. These elements may wrap texts across pages and there is no clear separation between pages. However there is a "page count" available in the metadata. Titles will often be wrapped in \<b\>. There is no size indication. Links are simply inline \<a\>. Sorted and unsorted list get a \<p class="list_Paragraph"\> per bullet point. However the count of the items in these lists are off in this particular example. Since PDF is a lot more structured than Word it might be interesting to convert Word to PDF and see what kind of structure that would yield.

#### 980c703f-1a5b-4ede-ab0d-ae73346df39c
Another PDF with similar structures as before. We could maybe train a transformer on PDF's similar to how we trained one for HTML, but we would need to make our own inferences about headers. Perhaps we can make a distinction between headers that start with a number and headers that do not.

#### a66199a5-3224-4de2-bb97-736d791506a2
A Microsoft Word Document. This one has more types of headers. Apart from the \<b\> wrapped in \<p\> there are also \<h1\> headers. This makes extracting titles a bit easier as well as trying to extract text in a similar way we do with the HTML transformer. I think we need a regex along the line of ``/^(.+)\d{0,2}:.+/``. If this matches inside header texts than the first group should go into our "common tags" (with lowercase). Otherwise the entire header can go in there.

#### b44e49ee-b7b8-4996-a417-bd5dda40894a
Very similar to 145cae9f-c834-4c38-8c34-4c76479da3ca

#### c6e37500-6421-4797-a118-fa381fff30f3
A Word document made with "Microsoft Word for the web". It has some good headers. The heading of the pages are also deduplicated and only present at the start of the document. This makes it easier to remove it from the main content. There's a TOC at the start with corresponding headers further down the document. So it should be possible to see this strucuture. The content itself is a bit more doubtful, but structure wise we might be able to clean it up a bit.

#### dd08ca16-7933-4790-8c06-14c491c92f96
First time I see a title in a PDF. It's made with a Mac, but the title claims to be Microsoft Word. It's going to be tough to extract headers from this document. It pattern seems to be: newline + title + newline without any markup surrounding it. Very similar to 1d863ed1-1609-453e-a6f3-5b5e107be543, but real Word documents like a66199a5-3224-4de2-bb97-736d791506a2 contain more clues than this PDF.

#### f0d3deb0-8a9e-424d-b8ee-0216d04f97e6
This is a bit the opposite of the last PDF. Headers and paragraphs don't occur in the same \<p\>, but every line is wrapped into it's own \<p\>. There is a very clear TOC with a numbering convention for the headers. So that seems like a way in to try to get some headers from a document like this. Should be possible to keep references and attachments out of the cleaned text if we work with some kind of vocabulary that flags these sections.

#### fb0c08c2-6bfc-43ee-b132-71cda7265964
Another Word document that has some TOC flagging with HTML classes. Makes it relatively easy to extract relevant titles. At the same time it's a document for a very specific course (including a time table). Such "materials" maybe shouldn't be part of the dataset, but I'll discuss that with Martine.